*3.1* Aggregate Quries

In [2]:
import sqlite3
import pandas as pd

In [3]:
# Connect to the SQLite database

conn =  sqlite3.connect("../database/db.sqlite3")

# Create a cursor object
cursor = conn.cursor()



In [12]:
#1. Student Analytics
# -------------------

# Count total number of students
count_query = f''' 
                SELECT COUNT(*) as total_students 
                FROM students;
                '''
count_result = pd.read_sql(count_query, conn)
print(count_result)

# use of 'as'

# In SQL, the "AS" keyword is used to rename a column or table in the result set of a 
# query. This is particularly useful for making the output more readable or for 
# simplifying column names in complex queries. 

# run the code without ' as total_student ' to see the difference 

   total_students
0               8


In [17]:
# Count students by department_id
sCount_query = f'''
                SELECT department_id, COUNT(*) as student_per_department
                FROM students
                GROUP BY department_id                
 '''
sCount_result = pd.read_sql(sCount_query,conn)
print(sCount_result)


   department_id  student_per_department
0              1                       4
1              2                       2
2              3                       2


> Now, you're probably thinking, 'What's this **GROUP BY** business all about?' Well, wonder no more! We're about to dive in and explore the wonderful world of **GROUP BY**!

>>The GROUP BY clause in SQL is used to group rows that have the same values in specified columns into aggregated data. It is often used in conjunction with aggregate functions like *COUNT()*, *SUM()*, *AVG()*, *MIN()*, and *MAX()* to perform calculations on each group of data.

**Rules**:
   - Columns in SELECT must be either:
     * Included in GROUP BY clause
     * Used with aggregate functions
   - Groups rows with matching values
   - Applied before HAVING clause